In [ ]:
import torch
import gc
import numpy as np
from PIL import Image
from torch import nn
from UNet import U_Net
from DataSource import train_dataloader,test_dataloader
model = U_Net(1,2)
model.train()
model = model.cuda()
#仿照指导书设置超参数，不设置validation set
learning_rate = 1e-1
epochs = 1
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#功力还不太够，先按照pytorch tutorials的流程来仿照着写
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        for times in range(500):
            X = X.cuda()
            y = y.cuda()
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        with torch.no_grad():
            count_of_img = 0
            pred = model(X)
            pred = pred.cpu()
            pred = nn.Softmax(dim=1)(pred)
            #pred = pred.int()
            pred = np.array(pred)
            #pred = np.where(pred>0.4,pred,0)
            pred = pred*255
            pred = pred.astype(np.int32)
            y = y.int()#直接转换会有问题，因为当初我们取label时为了使用交叉熵函数转成过Long型
            y = y.cpu()
            y = np.array(y)
            batch_size = pred.shape[0]
            for i in range(batch_size):
                tensor_this_time = pred[i][0]
                result_this_time = Image.fromarray(tensor_this_time)
                label_this_time = Image.fromarray(y[i]*255)
                result_this_time = result_this_time.convert("L")
                label_this_time = label_this_time.convert("L")
                result_this_time.show()
                result_this_time.save("result_"+str(count_of_img)+".jpg")
                label_this_time.save("label_"+str(count_of_img)+".jpg")
                count_of_img+=1
            """
            if batch % 10 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            """

def test_loop(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.cuda()
            y = y.cuda()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

#print(len(test_dataloader.dataset))
for t in range(epochs):
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    torch.save(model.state_dict(), 'model_weights_test.pth')
    test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 0.712611  [    0/16250]
loss: 1.618321  [    0/16250]
loss: 0.865546  [    0/16250]
loss: 0.710284  [    0/16250]
loss: 0.856164  [    0/16250]
loss: 0.760747  [    0/16250]
loss: 0.688979  [    0/16250]
loss: 0.733604  [    0/16250]
loss: 0.721833  [    0/16250]
loss: 0.706999  [    0/16250]
loss: 0.687057  [    0/16250]
loss: 0.698752  [    0/16250]
loss: 0.708584  [    0/16250]
loss: 0.706198  [    0/16250]
loss: 0.698851  [    0/16250]
loss: 0.690249  [    0/16250]
loss: 0.687285  [    0/16250]
loss: 0.691595  [    0/16250]
loss: 0.692959  [    0/16250]
loss: 0.691512  [    0/16250]
loss: 0.688586  [    0/16250]
loss: 0.685929  [    0/16250]
loss: 0.686166  [    0/16250]
loss: 0.687701  [    0/16250]
loss: 0.688505  [    0/16250]
loss: 0.688385  [    0/16250]
loss: 0.687626  [    0/16250]
loss: 0.686622  [    0/16250]
loss: 0.685724  [    0/16250]
loss: 0.685185  [    0/16250]
loss: 0.685109  [    0/16250]
loss: 0.685418  [    0/16250]
